In [41]:
import pandas as pd

#df = pd.read_csv('../data/donnees_nettoyees_avec_coord.csv', index_col=0)

df = pd.read_pickle('../data/dataframe_avec_coord.pkl')
print(len(df), " individus")


df['Code_postal'] = df['Code_postal'].astype(str)


# Créez une nouvelle colonne "Departement" contenant les 2 premiers chiffres du code postal
# gérer les 10 premiers départements
df['Departement'] = df['Code_postal'].apply(
    lambda x: '0' + x[:1] if len(x) == 4 else x[:2])

df['Departement'] = df['Departement'].astype(str)
# BUG AVEC LA CORSE CAR DÉPARTEMENTS = 2A etc... ????

print(df['Departement'].nunique()," départements différents")
df.head(15)




743441  individus
28  départements différents


,No_disposition,Date_mutation,Nature_mutation,Valeur_fonciere,Code_postal,Commune,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Surface_terrain,year,latitude,longitude,Departement
0,1,2018-05-07,Vente,163000.0,11120,MOUSSAN,0,1,65.0,2.0,S,575.0,2018,43.230595,2.956356,11
1,1,2018-04-27,Vente,94700.0,11600,ARAGON,0,1,143.0,4.0,S,39.0,2018,43.298767,2.307072,11
2,1,2018-07-02,Vente,40000.0,10340,LES RICEYS,0,3,0.0,0.0,S,36.0,2018,47.987488,4.369501,10
3,1,2018-04-04,Vente,159400.0,13190,ALLAUCH,1,4,20.0,0.0,NaN,NaN,2018,43.353315,5.511348,13
4,1,2018-06-20,Vente,75000.0,5170,ORCIERES,2,2,52.0,2.0,NaN,NaN,2018,44.684098,6.348975,05
5,1,2018-03-22,Vente,160000.0,2400,CHATEAU THIERRY,0,1,94.0,4.0,S,260.0,2018,49.056407,3.381600,02
6,1,2018-09-13,Vente,238980.0,6000,NICE,1,2,41.0,2.0,NaN,NaN,2018,43.711999,7.238269,06
7,1,2018-02-09,Vente,48500.0,1100,OYONNAX,2,2,70.0,4.0,NaN,NaN,2018,46.260544,5.653443,01
8,1,2018-12-07,Vente,70000.0,13200,ARLES,0,1,64.0,2.0,S,44.0,2018,43.546869,4.662156,13
9,1,2018-01-30,Vente,145000.0,6000,NICE,2,2,52.0,3.0,NaN,NaN,2018,43.711999,7.238269,06


In [42]:
import folium
from folium.plugins import HeatMap
from branca.colormap import linear

# Créez une liste de listes contenant les coordonnées [latitude, longitude] et la valeur "Valeur fonciere"
heat_data = [[row['latitude'], row['longitude'], row['Valeur_fonciere']]
             for index, row in df.iterrows()]

# Créez la carte
my_map = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Créez une colormap linéaire basée sur la colonne "Valeur fonciere"
colormap = linear.YlOrRd_09.scale(
    min(df['Valeur_fonciere']), max(df['Valeur_fonciere']))

# Ajoutez la carte de chaleur à partir des données
HeatMap(heat_data,
        min_opacity=0.2,
        max_val=max(df['Valeur_fonciere']),
        radius=15,
        blur=10,
        max_zoom=1,
        gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'}).add_to(my_map)

# Ajoutez une légende basée sur la colormap
colormap.caption = "Valeur fonciere"
colormap.add_to(my_map)

# Enregistrez la carte en tant que fichier HTML
my_map.save('map.html')


/tmp/ipykernel_2764/449697079.py:17: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(heat_data,


In [43]:
import folium
from folium.plugins import HeatMap
from folium import Choropleth, GeoJson
import pandas as pd

# Calcul de la valeur foncière moyenne par département
df_avg_value_by_dept = df.groupby(
    'Departement')['Valeur_fonciere'].mean().reset_index()

print(df_avg_value_by_dept)
print(len(df_avg_value_by_dept), " départements différents")
# Coordonnées du centre de la France (ou un autre emplacement de votre choix)
latitude, longitude = 46.603354, 1.888334

# Créez la carte
my_map = folium.Map(location=[latitude, longitude], zoom_start=6)

# Ajoutez la carte choroplèthe en fonction de la valeur foncière moyenne par département
Choropleth(
    # Chemin vers le fichier GeoJSON des départements de la France
    geo_data='../data/departements.geojson',
    name='choropleth',
    data=df_avg_value_by_dept,
    columns=['Departement', 'Valeur_fonciere'],
    # Clé pour faire correspondre les données GeoJSON avec le DataFrame
    key_on='feature.properties.code',
    # Palette de couleurs (vous pouvez choisir d'autres palettes)
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Valeur fonciere moyenne par département'
).add_to(my_map)

# Enregistrez la carte en tant que fichier HTML
my_map.save('map_departements.html')


   Departement  Valeur_fonciere
0           01    265251.035434
1           02    226510.784212
2           03    340276.378787
3           04    161946.570753
4           05    156519.542746
5           06    338756.934976
6           07    172987.675816
7           08    117863.965137
8           09    122936.163258
9           10    144615.423018
10          11    140895.996408
11          12    128949.521795
12          13    350088.295736
13          14    886094.852869
14          15    111116.630852
15          16    142475.416409
16          17    238601.410993
17          18    295657.356751
18          19    119488.815203
19          21    174790.460569
20          22    434555.634094
21          23     79844.186005
22          24    167151.674710
23          25    152268.711599
24          33    153427.083333
25          42     91932.000000
26          43     16833.333333
27          48     58204.000000
28  départements différents


In [44]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="my_geocoder")

# Remplacez le code postal correct de la ville voulue et vérifier les coordonnées
code_postal = "2200"
adresse = f"SOISSONS, {code_postal}, France"

# Utilisez Nominatim pour obtenir les coordonnées géographiques
location = geolocator.geocode(adresse)

# Affichez les coordonnées géographiques (latitude et longitude) de "AVERMES"
if location:
    print(f"Latitude: {location.latitude}, Longitude: {location.longitude}")
else:
    print("Coordonnées non trouvées pour l'adresse spécifiée.")


Latitude: 49.3837621, Longitude: 3.327613
